## Probemos el modelo de spaCy para NER en español.

Esto podría servir para eliminar de los títulos palabras que para el caso particular de nuestro problema no aportan información a la clasificación, como adjetivos, artículos, etc. También viene un modelo para portugués.

In [0]:
! python -m spacy download es_core_news_sm

     |████████████████████████████████| 16.2MB 2.6MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=e0ccf0ebdf5028b33c931fac289e8dbd43977e4bb62ef6c0ce0c65c310cef621
  Stored in directory: /tmp/pip-ephem-wheel-cache-p5yfm7z3/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [0]:
import es_core_news_sm as spacy 

In [0]:
nlp = spacy.load()  

In [0]:
sent = nlp("Esto es una frase.")
print([(w.text, w.pos_) for w in sent])
# acá estoy viendo la 'Part Of Speech' tag

[('Esto', 'PRON'), ('es', 'AUX'), ('una', 'DET'), ('frase', 'NOUN'), ('.', 'PUNCT')]


In [0]:
print([(w.text, w.tag_) for w in sent])
# acá me muestra tags más específicas, por ej el modo, el género, el número, etc. Abuso.

[('Esto', 'PRON__Number=Sing|PronType=Dem'), ('es', 'AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin'), ('una', 'DET__Definite=Ind|Gender=Fem|Number=Sing|PronType=Art'), ('frase', 'NOUN__Gender=Fem|Number=Sing'), ('.', 'PUNCT__PunctType=Peri')]


spaCy tiene un visualizador de las dependencias de las componentes de la oración. Veamos qué onda.

In [0]:
from spacy import displacy
displacy.render(sent, style="dep", jupyter=True)

Me acabo de dar cuenta que en este ejemplo no identificó a 'es' como un verbo. Eso es medio un moco pero para nuestro problema no sería tan molesto, siempre y cuando no nos meta un verbo como sustantivo. Veamos otra frase.

In [0]:
senten = nlp("Harry se despertó con un sobresalto")
displacy.render(senten, style="dep", jupyter=True)

In [0]:
doc = nlp('Se trata de un ave de tamaño medio que mide 14 cm de longitud. Al igual que otros trepadores, es un pájaro robusto, con cola corta y largo pico. Las partes superiores son de color gris azulado, los colores de las partes inferiores varían según su ubicación geográfica del rojo puro al blanco o amarillento, con o sin un área clara en la mejilla.')
print([(w.text, w.pos_) for w in doc])

[('Se', 'PRON'), ('trata', 'VERB'), ('de', 'ADP'), ('un', 'DET'), ('ave', 'NOUN'), ('de', 'ADP'), ('tamaño', 'NOUN'), ('medio', 'ADJ'), ('que', 'PRON'), ('mide', 'VERB'), ('14', 'NUM'), ('cm', 'NOUN'), ('de', 'ADP'), ('longitud', 'NOUN'), ('.', 'PUNCT'), ('Al', 'ADP'), ('igual', 'ADJ'), ('que', 'SCONJ'), ('otros', 'DET'), ('trepadores', 'NOUN'), (',', 'PUNCT'), ('es', 'AUX'), ('un', 'DET'), ('pájaro', 'NOUN'), ('robusto', 'ADJ'), (',', 'PUNCT'), ('con', 'ADP'), ('cola', 'NOUN'), ('corta', 'PROPN'), ('y', 'CONJ'), ('largo', 'ADJ'), ('pico', 'ADJ'), ('.', 'PUNCT'), ('Las', 'DET'), ('partes', 'NOUN'), ('superiores', 'ADJ'), ('son', 'AUX'), ('de', 'ADP'), ('color', 'NOUN'), ('gris', 'ADJ'), ('azulado', 'ADJ'), (',', 'PUNCT'), ('los', 'DET'), ('colores', 'NOUN'), ('de', 'ADP'), ('las', 'DET'), ('partes', 'NOUN'), ('inferiores', 'ADJ'), ('varían', 'VERB'), ('según', 'ADP'), ('su', 'DET'), ('ubicación', 'NOUN'), ('geográfica', 'ADJ'), ('del', 'ADP'), ('rojo', 'NOUN'), ('puro', 'ADJ'), ('al', 

Bueno spaCy tiene muchísimos [TOKEN attibutes](https://spacy.io/api/token#attributes) a los que podemos acceder. El .pos_, por ej, nos tira una etiqueta 'gruesa' (coarse) describiendo qué función tiene esa palabra en la oración (i.e. si es un verbo, un susantivo, un adjetivo, etc) mientras que el .tag_ ya es algo más específico, tirando una etiqueta  'fina' que no sólo describe la función de la palabra sino también el modo, el tiempo verbal, el género, el número, etc. Es fantástico. 

La lista de las .pos_ está [acá](https://spacy.io/api/annotation#pos-tagging). De todas formas hay que tener cuidado porque por ejemplo cosas como 'cm' se clasifican como sustantivos (en este caso al menos) y no estaríamos necesitando ese tipo de sustantivo, habría que hacer una limpieza antes. También veo que el adjetivo 'corta' se clasificó como nombre propio, andá a saber por qué. También reconoce los signos de puntuación, pero de todas formas yo capaz le sacaría todos los caracteres no alfanuméricos.

A ver cómo seleccionaría sólo los sutantivos, por ej.

In [18]:
for w in doc:
  if w.pos_ in ['NOUN', 'PROPN']:
    print(w.text)

ave
tamaño
cm
longitud
trepadores
pájaro
cola
corta
partes
color
colores
partes
ubicación
rojo
blanco
amarillento
área
mejilla


Quiero ver si cambia la label que le pone a las palabras si le saco los signos de puntuación.

In [0]:
import re
frase = 'En la cima de una alta montaña, en el otro costado, con sus ventanas brillando en el cielo estrellado, había un enorme castillo con muchas torres y atalayas.'
f_punt = nlp(frase)
f_nopunt = nlp(re.sub('[^A-Za-z0-9-ñ]+', ' ', frase))

In [27]:
print([(w.text, w.pos_) for w in f_punt])
print([(w.text, w.pos_) for w in f_nopunt])

[('En', 'ADP'), ('la', 'DET'), ('cima', 'NOUN'), ('de', 'ADP'), ('una', 'DET'), ('alta', 'ADJ'), ('montaña', 'NOUN'), (',', 'PUNCT'), ('en', 'ADP'), ('el', 'DET'), ('otro', 'DET'), ('costado', 'NOUN'), (',', 'PUNCT'), ('con', 'ADP'), ('sus', 'DET'), ('ventanas', 'NOUN'), ('brillando', 'VERB'), ('en', 'ADP'), ('el', 'DET'), ('cielo', 'NOUN'), ('estrellado', 'ADJ'), (',', 'PUNCT'), ('había', 'AUX'), ('un', 'DET'), ('enorme', 'ADJ'), ('castillo', 'NOUN'), ('con', 'ADP'), ('muchas', 'DET'), ('torres', 'NOUN'), ('y', 'CONJ'), ('atalayas', 'NOUN'), ('.', 'PUNCT')]
[('En', 'ADP'), ('la', 'DET'), ('cima', 'NOUN'), ('de', 'ADP'), ('una', 'DET'), ('alta', 'ADJ'), ('montaña', 'NOUN'), ('en', 'ADP'), ('el', 'DET'), ('otro', 'DET'), ('costado', 'NOUN'), ('con', 'ADP'), ('sus', 'DET'), ('ventanas', 'NOUN'), ('brillando', 'VERB'), ('en', 'ADP'), ('el', 'DET'), ('cielo', 'NOUN'), ('estrellado', 'ADJ'), ('hab', 'PROPN'), ('a', 'ADP'), ('un', 'DET'), ('enorme', 'ADJ'), ('castillo', 'NOUN'), ('con', 'ADP

In [28]:
%%time
f_punt = nlp(frase)

CPU times: user 22.2 ms, sys: 0 ns, total: 22.2 ms
Wall time: 25.4 ms
